<a href="https://colab.research.google.com/github/amazingashis/Hackfest/blob/master/tuberculosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.style.use('fivethirtyeight')
import pickle 
import os 
import numpy as np
import cv2 
%matplotlib inline
import pandas.util.testing as tm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [4]:
labels = ['Yes', 'No']
img_size = 200
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [5]:
data = get_training_data('/content/drive/MyDrive/Data/data/Tuber-data')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [6]:
X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)



# resize data for deep learning 
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [7]:
X_train = X_train / 255
X_test = X_test / 255

In [8]:
# good for balancing out disproportions in the dataset 
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=90, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True)  

datagen.fit(X_train)

In [9]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(200,200,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(2, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 15

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 99, 99, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 78408)             0         
_________________________________________________________________
Softmax (Dense)              (None, 2)                 156818    
Total params: 156,898
Trainable params: 156,898
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
6/6 [==============================] - 1s 64ms/step - loss: 2.9981 - accuracy: 0.4955
Epoch 2/15
6/6 [==============================] - 0s 63ms/step - loss: 1.4030 - accuracy: 0.5100
Epoch 3/15
6/6 [==============================] - 0s 65ms/step - loss: 1.0550 - accuracy: 0.4846
Epoch 4/15
6/6 [==============================] - 0s 64ms/step - loss: 0.8007 - accuracy:

In [13]:
print(model.predict(X_test))

[[0.91215384 0.08784613]
 [0.7986131  0.20138685]
 [0.33259463 0.66740537]
 [0.08491125 0.9150888 ]
 [0.5891039  0.41089615]
 [0.3393489  0.6606511 ]
 [0.43723246 0.56276757]
 [0.79458463 0.20541537]
 [0.02829581 0.97170424]
 [0.01407258 0.98592746]
 [0.25851336 0.7414866 ]
 [0.8754203  0.12457977]
 [0.88788426 0.11211568]
 [0.14011844 0.8598816 ]
 [0.3266466  0.6733534 ]
 [0.71510804 0.28489193]
 [0.12970413 0.8702959 ]
 [0.85513383 0.14486621]
 [0.9439561  0.05604391]
 [0.25779554 0.7422045 ]
 [0.09344793 0.906552  ]
 [0.933435   0.06656497]
 [0.6715925  0.32840753]
 [0.07010552 0.9298945 ]
 [0.8102596  0.18974033]
 [0.19723807 0.802762  ]
 [0.83368164 0.16631842]
 [0.13610606 0.8638939 ]
 [0.9317803  0.06821973]
 [0.14236249 0.85763747]
 [0.9348065  0.06519352]
 [0.21001719 0.7899828 ]
 [0.13122946 0.8687705 ]
 [0.91115326 0.08884677]
 [0.35620585 0.6437941 ]
 [0.92841136 0.07158867]
 [0.84428024 0.15571982]
 [0.03219889 0.96780115]
 [0.93446386 0.06553612]
 [0.37871334 0.6212867 ]


In [11]:
!pip install tensorflowjs

INFO:tensorflow:Assets written to: /tmp/1613842481/assets


In [16]:
import time
saved_model_path = "tmp/{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [17]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2021-02-20 17:35:56.127748: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
